In [1]:
import pandas as pd
import numpy as np
import random 

import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By

from lib.geo_to_vector import vectorize_geo

In [2]:
df = pd.read_csv("cleaned_complete.csv")
df.shape

(13048, 22)

In [3]:
vectorize_geo(df)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\matan\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\matan\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\matan\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


,Unnamed: 0,company_name,company_about,founded,business model,employees,funding stage,raised,product stage,status,...,total_rounds,investors,ipo_price,tag_vec,targetmarket_vec,sector_vec,target_ind_vec,tech_vec,suceeded,geo_market_per
0,0,Tastewise,Tastewise is an AI platform designed to help f...,7/2017,0,51-200,ROUND A,21500000.0,Released,1,...,3,3,0.0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,0.239990
1,1,Wilk Technologies,Wilk is dedicated to revolutionizing the dairy...,6/2018,1,11-50,Public,4690000.0,R&D,1,...,2,2,0.0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, ...","[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,NaN
2,2,Eco Pack Green Box,Eco Pack Green Box has developed and patented ...,3/2008,0,11-50,Revenue Financed,0.0,Released,0,...,0,0,0.0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,0.068500
3,3,BeeHero,BeeHero has developed a platform that can pred...,10/2017,0,1-10,ROUND A,24000000.0,Released,1,...,4,12,0.0,"[0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,1.000000
4,4,Cham Foods,Cham Foods is a multinational company with man...,12/1970,1,11-50,Public,0.0,Released,1,...,0,0,0.0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13043,13043,TriEye,TriEye is a fabless semiconductor company that...,11/2017,0,51-200,ROUND A,96000000.0,R&D,1,...,4,12,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1.0,1.000000
13044,13044,LYNX Smartcars,LYNX is developing software for connected and ...,1/2016,0,1-10,Bootstrapped,0.0,R&D,0,...,0,0,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...",0.0,NaN
13045,13045,Deeyook Location Technologies,Deeyook seeks to redefine location technology ...,3/2017,0,11-50,Seed,0.0,Released,1,...,0,0,0.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0,1.000000
13046,13046,SafeCue,SafeCue combines the power of deep learning wi...,1/2016,0,1-10,Seed,500000.0,Beta,0,...,1,1,0.0,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.0,0.819272


In [ ]:
def get_driver():
    from selenium import webdriver
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.chrome.service import Service
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    
    return driver

In [ ]:
def listify_geo_markets(df):
    l = []
    for i,val in enumerate(df['geographical markets']):
        if val is not np.nan:
            l.append(val.split(', '))
        else :
            l.append(np.nan)
    return l

In [ ]:
def get_location_list(df: pd.DataFrame)-> list : 
    countries = []
    for list in df['geographical markets']:
        if list is not np.nan:
            for country in list:
                if country not in countries:
                    countries.append(country.lower())
    
    return countries

In [ ]:
# def get_api_data(countries):
#     api = "https://restcountries.com/v3.1/"
#     api_v = ['name/','region/','subregion/']

#     d = {}
#     for country in countries:
#         time.sleep(0.5)
#         for v in api_v:
#             r = requests.get(f"{api}{v}{country}")
#             if r.status_code == 200:
#                 d.update({country : r.json()[0]['population']})
#                 break
    
#     return d


In [ ]:
def get_world_pop() -> int :
    """Get world population by scraping www.worldometers.info"""
    
    driver = get_driver()

    url = "https://www.worldometers.info/world-population/"
    driver.get(url)
    population = driver.find_element(By.CLASS_NAME, "maincounter-number").text
    driver.quit()
    pop = population.split(',')
    num = ''.join(pop)
    return int(num)



In [ ]:
def get_countries_population() -> dict:
    """Get countries population by scraping www.worldometers.info"""
    
    
    driver = get_driver()
    url = "https://www.worldometers.info/world-population/population-by-country/"
    driver.get(url)
    d = {}
    table = driver.find_element(By.TAG_NAME, "tbody").find_elements(By.TAG_NAME, "tr")

    for row in table:
        elements = row.find_elements(By.TAG_NAME, "td")
        d.update({elements[1].text.lower(): elements[2].text.lower()})
    
    driver.quit()
    return d
    

In [ ]:
df['geographical markets'] = listify_geo_markets(df)

In [ ]:
countries = get_location_list(df)
countries

In [ ]:
country_pop = get_countries_population()

In [ ]:
dict_update = {'palestine':'State of Palestine','macedonia':'North Macedonia','myanmar (burma)':'myanmar',
"cote d'ivoire":"côte d'ivoire", 'czech republic':'Czech Republic (Czechia)', 'swaziland': 'eswatini'}

for key,val in dict_update.items():
    country_pop.update({key.lower():country_pop[val.lower()]})

country_pop.update({'global' : get_world_pop()})
country_pop.update({'north asia' : country_pop['russia']})
country_pop

In [ ]:
def get_not_in_list_data(l: list):
    d = {}
    driver = get_driver()
    url = "https://www.worldometers.info/world-population/"  #  "western-europe-population/"
    postfix=['','ern']
    americas_population = 0

    for country in l:

        if country == 'americas':
            driver.get(f'{url}latin-america-and-the-caribbean-population')
            time.sleep(random.uniform(0.5,1.5))
            americas_population = int(''.join(driver.find_element(By.CLASS_NAME, "maincounter-number").text.split(',')))
            time.sleep(random.uniform(5,8))
            driver.get(f'{url}northern-america-population')
            time.sleep(random.uniform(0.5,1.5))
            americas_population += int(''.join(driver.find_element(By.CLASS_NAME, "maincounter-number").text.split(',')))

        elif country == 'southeast asia':
            driver.get(f'{url}south-eastern-asia-population')
            population = int(''.join(driver.find_element(By.CLASS_NAME, "maincounter-number").text.split(',')))
            d.update({country: population})

        elif country.count(' ') == 0:
            try:
                new_url = f"{url}{country}-population"
                driver.get(new_url)
                time.sleep(random.uniform(0.5,1.5))
                population = int(''.join(driver.find_element(By.CLASS_NAME, "maincounter-number").text.split(',')))
                if population is not None:
                    d.update({country: population})
                    continue
                    
            except Exception as e:
                pass
                # print("Error on "+ new_url)
                # print(str(e))
        else:
            sp_index = country.rindex(' ')
            for p in postfix:
                try:
                    newcountry = f"{country[:sp_index]}{p}{country[sp_index:]}"
                    new_url = url+ "-".join(newcountry.split(' '))+'-population'
                    driver.get(new_url)
                    time.sleep(random.uniform(0.5,1.5))
                    population = int(''.join(driver.find_element(By.CLASS_NAME, "maincounter-number").text.split(',')))
                    if population is not None:
                        d.update({country: population})
                        break
                    
                except:
                    # print("Error on "+ new_url)
                    continue
        
        time.sleep(random.uniform(5,8))
            

        
    # Special treatment for americas
    if americas_population != 0:
        d.update({'americas': americas_population})

    # time.sleep(10)
    driver.quit()
    return d


In [ ]:
# List definistions
not_in_list = []

for country in countries:
    if country not in country_pop.keys():
        not_in_list.append(country)


In [ ]:
print(f"dictionary length before: {len(country_pop)}")
country_pop.update(get_not_in_list_data(not_in_list))
print(f"dictionary length after: {len(country_pop)}")


In [ ]:
world_population = country_pop['global']
country_pop_percent = {}
for key,val in country_pop.items():

    if type(val) == str:
        popu = val.split(',')
        country_pop_percent.update({key : int(''.join(popu)) / world_population})
    elif type(val) == int:
        country_pop_percent.update({key : val / world_population})

country_pop_percent

In [ ]:
country_pop_percent['north america']

In [ ]:
for i, list in enumerate(df['geographical markets']):
    sum = 0
    if list is not np.nan :
        for country in list :
            if country in country_pop_percent.keys() :
                sum += country_pop_percent[country]
        df.loc[i,'geo_market_per'] = sum if sum < 1 else 1


df['geo_market_per']

